In [1]:
!pip install ultralytics --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.6 MB/s eta 0:00:00


In [2]:
!pip install fiftyone --quiet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.8/112.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 121.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 91.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.5/934.5 kB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 15.5 MB/s eta 

In [3]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
import cv2
import yaml
import random
import os
from ultralytics.utils.downloads import download
import kagglehub
from IPython.display import Image, display
from pathlib import Path
import json
from tqdm import tqdm

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
model = YOLO('/content/drive/MyDrive/YOLOv5/coco_finetune/weights/last.pt')

path = kagglehub.dataset_download("awsaf49/coco-2017-dataset")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'coco-2017-dataset' dataset.
Path to dataset files: /kaggle/input/coco-2017-dataset


In [6]:
print("Folders in dataset:", os.listdir(path))

annot_path = os.path.join(path, 'coco2017')
if os.path.exists(annot_path):
    print("Annotation files:", os.listdir(annot_path))

Folders in dataset: ['coco2017']
Annotation files: ['val2017', 'annotations', 'test2017', 'train2017']


In [7]:
def convert_coco_json(json_path, output_label_dir):

    # Mapping lables since we want to convert them to .txt format for yolo to read
    mapping = {1:0, 3:1, 8:2, 6:3, 2:4, 4:5, 10:6, 11:7, 13:8}

    # Making a folder to save the lables since we're creating a directory (images -> train, val | lables -> train, val)
    os.makedirs(output_label_dir, exist_ok=True)

    # coverting coco json to python dictionary to search easily
    with open(json_path, 'r') as f:
        data = json.load(f)

    # creating dictionary where key is image id and value is image data
    images = {img['id']: img for img in data['images']}

    print(f"Processing {len(data['annotations'])} annotations...")

    # loop on all annotations
    for ann in tqdm(data['annotations']):
        cat_id = ann['category_id']
        #checking if category id in mapping
        if cat_id in mapping:
            img_id = ann['image_id']
            img_info = images[img_id]
            w, h = img_info['width'], img_info['height']

            x_min, y_min, box_w, box_h = ann['bbox']
            x_c = (x_min + box_w / 2) / w # X center
            y_c = (y_min + box_h / 2) / h # Y center
            wn = box_w / w # normalized width
            hn = box_h / h # normalized height

            txt_name = Path(img_info['file_name']).stem + ".txt"
            with open(os.path.join(output_label_dir, txt_name), 'a') as f:
                f.write(f"{mapping[cat_id]} {x_c:.6f} {y_c:.6f} {wn:.6f} {hn:.6f}\n")

In [8]:
base_dir = Path('/content/yolo_dataset')
# Dataset Arrnaged
dirs = {
    'train_images': base_dir / 'final/images/train',
    'val_images': base_dir / 'final/images/val',
    'train_labels': base_dir / 'final/labels/train',
    'val_labels': base_dir / 'final/labels/val',
}

for dir_path in dirs.values():
    dir_path.mkdir(parents=True, exist_ok=True)

print("Directory structure created!")
print(f"Base directory: {base_dir}")
print(f"{dirs}")

Directory structure created!
Base directory: /content/yolo_dataset
{'train_images': PosixPath('/content/yolo_dataset/final/images/train'), 'val_images': PosixPath('/content/yolo_dataset/final/images/val'), 'train_labels': PosixPath('/content/yolo_dataset/final/labels/train'), 'val_labels': PosixPath('/content/yolo_dataset/final/labels/val')}


In [9]:
# Coverted from coco format to yolo format (only coverting labels from json to .txt)
convert_coco_json(
    f"{path}/coco2017/annotations/instances_train2017.json",
    dirs['train_labels']
)

convert_coco_json(
    f"{path}/coco2017/annotations/instances_val2017.json",
    dirs['val_labels']
)

Processing 860001 annotations...


100%|██████████| 860001/860001 [00:14<00:00, 57993.16it/s]


Processing 36781 annotations...


100%|██████████| 36781/36781 [00:00<00:00, 52648.27it/s]


In [10]:
# Linking the images here
coco_train_src = os.path.join(path, 'coco2017/train2017')
coco_val_src = os.path.join(path, 'coco2017/val2017')

for img in os.listdir(coco_train_src):
    src = os.path.join(coco_train_src, img)
    dst = os.path.join(dirs['train_images'], img)
    if not os.path.exists(dst):
        os.symlink(src, dst)

for img in os.listdir(coco_val_src):
    src = os.path.join(coco_val_src, img)
    dst = os.path.join(dirs['val_images'], img)
    if not os.path.exists(dst):
        os.symlink(src, dst)

print("images and labels are linked and are side-by-side.")

images and labels are linked and are side-by-side.


In [11]:
from google.colab import drive
drive.mount('/content/drive')

data_config = {
    'path': '/content/yolo_dataset/final',
    'train': 'images/train',
    'val': 'images/val',
    'nc': 9,
    'names': [
        'person', 'car', 'truck', 'bus', 'bicycle',
        'motorcycle', 'traffic_light', 'speed_limit', 'stop'
    ]
}

save_dir = '/content/drive/MyDrive/YOLOv5/datasets/coco_finetune'
yaml_path = os.path.join(save_dir, 'data.yaml')

#os.makedirs(save_dir, exist_ok=True)

with open(yaml_path, 'w') as f:
    yaml.dump(data_config, f, default_flow_style=False)

print(f'Data config path: {data_config}')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Data config path: {'path': '/content/yolo_dataset/final', 'train': 'images/train', 'val': 'images/val', 'nc': 9, 'names': ['person', 'car', 'truck', 'bus', 'bicycle', 'motorcycle', 'traffic_light', 'speed_limit', 'stop']}


In [ ]:
# model.train(
#     data='/content/drive/MyDrive/YOLOv5/datasets/coco_finetune/data.yaml',
#     epochs=50,
#     imgsz=640,
#     batch=16,
#     device=0,
#     workers=4,
#     project='/content/drive/MyDrive/YOLOv5',
#     name='coco_finetune',
#     save_period = 1,
#     exist_ok=True
# )



model.train(data='/content/drive/MyDrive/YOLOv5/datasets/coco_finetune/data.yaml',resume=True,name='coco_finetune',project='/content/drive/MyDrive/YOLOv5',save_period = 1, exist_ok=True)

metrics = model.val()
print(f"Mean Average Precision (mAP): {metrics.box.map}")

Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.10.0+cu128 CUDA:0 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/YOLOv5/datasets/coco_finetune/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/drive/MyDrive/YOLOv5/coco_finetune/weights/last.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=coco_finetune, nbs=64, nms=False, op

In [ ]:
# Look at the first entry in the annotations list
# sample_ann = data['annotations'][0]

# import pprint
# pp = pprint.PrettyPrinter(indent=4)
# pp.pprint(sample_ann)